# Домашнее задание

## Функции в Pandas

**Задание 1**


Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
	- оценка 2 и меньше - низкий рейтинг
	- оценка 4 и меньше - средний рейтинг
	- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [1]:
import pandas as pd

In [2]:
def assess_rating(row):
    if row <= 2:
        return 'низкий рейтинг'
    if row <= 4:
        return 'средний рейтинг'
    return 'высокий рейтинг'

In [3]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
joined = ratings.merge(pd.read_csv('ml-latest-small/movies.csv'), on = 'movieId', how = 'left')
joined.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [4]:
joined['class'] = joined['rating'].apply(assess_rating)
joined.head()

,userId,movieId,rating,timestamp,title,genres,class
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,средний рейтинг
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance,средний рейтинг
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller,средний рейтинг
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,высокий рейтинг
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,высокий рейтинг


In [6]:
joined['class'].value_counts()

средний рейтинг    65551
высокий рейтинг    21762
низкий рейтинг     13523
Name: class, dtype: int64

**Задание 2**


Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

Результат классификации запишите в отдельный столбец region.

In [7]:
import re

In [8]:
data = pd.read_csv('keywords.csv')
data.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [9]:
geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

geo_pattern = {r: '|'.join([(city[:-1] + '[а-я]+') for city in c]) for r,c in geo_data.items()}
print(geo_pattern)

{'Центр': 'москв[а-я]+|тул[а-я]+|ярославл[а-я]+', 'Северо-Запад': 'петербур[а-я]+|пско[а-я]+|мурманс[а-я]+', 'Дальний Восток': 'владивосто[а-я]+|сахали[а-я]+|хабаровс[а-я]+'}


In [10]:
def find_region(row):
    reg = [r for r,p in geo_pattern.items() if re.search(p, row, flags=re.IGNORECASE)] or ['undefined']
    return reg[0]

In [11]:
data['region'] = data['keyword'].apply(find_region)
data.region.value_counts()

# если городов несколько (сапсан москва санкт-петербург), ищет первый попавшийся в geo_data (москва)

undefined         98787
Центр               854
Северо-Запад        265
Дальний Восток       94
Name: region, dtype: int64

**Задание 3 (бонусное)**


Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
	- для каждой строки пройдите по всем годам списка years
	- если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
	- если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга

In [12]:
joined = pd.read_csv('ml-latest-small/ratings.csv').merge(pd.read_csv('ml-latest-small/movies.csv'), on = 'movieId', how = 'left')

In [13]:
# step 1
years = [str(y) for y in range(1950, 2010 + 1)]

In [14]:
# step 2
def production_year(title):
    year = [year for year in years if year in title] or ['1990']
    if len(year) > 1:
        print(title, year)  # это можно убрать - нужно только для проверки логики
    return year[0]

# не очень хорошо работает - для названий типа "2001: A Space Odyssey (1968)", "The Godfather Trilogy: 1972-1990 (1992)",
# "2001 Maniacs (2005)", "Godzilla 2000 (Gojira ni-sen mireniamu) (1999)" получается ерунда 

In [15]:
# step 3
joined['year'] = joined['title'].apply(production_year)

Death Race 2000 (1975) ['1975', '2000']
2001: A Space Odyssey (1968) ['1968', '2001']
2001: A Space Odyssey (1968) ['1968', '2001']
2001: A Space Odyssey (1968) ['1968', '2001']
Fantasia 2000 (1999) ['1999', '2000']
2001: A Space Odyssey (1968) ['1968', '2001']
2001: A Space Odyssey (1968) ['1968', '2001']
2010: The Year We Make Contact (1984) ['1984', '2010']
2001: A Space Odyssey (1968) ['1968', '2001']
2001: A Space Odyssey (1968) ['1968', '2001']
Blues Brothers 2000 (1998) ['1998', '2000']
2001: A Space Odyssey (1968) ['1968', '2001']
2001: A Space Odyssey (1968) ['1968', '2001']
2001: A Space Odyssey (1968) ['1968', '2001']
2010: The Year We Make Contact (1984) ['1984', '2010']
2001: A Space Odyssey (1968) ['1968', '2001']
2010: The Year We Make Contact (1984) ['1984', '2010']
2001: A Space Odyssey (1968) ['1968', '2001']
2001: A Space Odyssey (1968) ['1968', '2001']
2001: A Space Odyssey (1968) ['1968', '2001']
2010: The Year We Make Contact (1984) ['1984', '2010']
Fantasia 2000 

In [16]:
joined.head()

,userId,movieId,rating,timestamp,title,genres,year
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance,1995
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller,1995
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,1995
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,1995


In [17]:
# step 4
joined.groupby('year').agg({'rating': 'mean'}).sort_values('rating', ascending = False)

,rating
year,
1957,4.039535
1954,4.009191
1962,3.969466
1952,3.953125
1972,3.944293
...,...
2000,3.398922
2005,3.359976
1992,3.353555


**Вариант 2 - с регулярным выражением**

In [18]:
import re

In [19]:
joined = pd.read_csv('ml-latest-small/ratings.csv').merge(pd.read_csv('ml-latest-small/movies.csv'), on = 'movieId', how = 'left')

In [20]:
def production_year(title):
    pattern = '[\(\-](\d{4})\)'
    year = re.findall(pattern, title) or ['1900']
    
    # следующие 2 строки только для проверки логики
    if title in ['2001: A Space Odyssey (1968)', '2001 Maniacs (2005)', 'Godzilla 2000 (Gojira ni-sen mireniamu) (1999)', 'The Godfather Trilogy: 1972-1990 (1992)']:
        print(title, f'year = {year[0]}')
    
    if int(year[0]) in range(1950, 2010 + 1):
            return year[0]
    return '1900'

In [21]:
joined['year'] = joined['title'].apply(production_year)

# так год выбирается аккуратнее

2001: A Space Odyssey (1968) year = 1968
2001: A Space Odyssey (1968) year = 1968
2001: A Space Odyssey (1968) year = 1968
2001: A Space Odyssey (1968) year = 1968
2001: A Space Odyssey (1968) year = 1968
2001: A Space Odyssey (1968) year = 1968
2001: A Space Odyssey (1968) year = 1968
2001: A Space Odyssey (1968) year = 1968
2001: A Space Odyssey (1968) year = 1968
2001: A Space Odyssey (1968) year = 1968
2001: A Space Odyssey (1968) year = 1968
2001: A Space Odyssey (1968) year = 1968
2001: A Space Odyssey (1968) year = 1968
2001: A Space Odyssey (1968) year = 1968
2001: A Space Odyssey (1968) year = 1968
The Godfather Trilogy: 1972-1990 (1992) year = 1992
2001: A Space Odyssey (1968) year = 1968
2001: A Space Odyssey (1968) year = 1968
2001: A Space Odyssey (1968) year = 1968
2001: A Space Odyssey (1968) year = 1968
2001: A Space Odyssey (1968) year = 1968
2001: A Space Odyssey (1968) year = 1968
2001: A Space Odyssey (1968) year = 1968
2001: A Space Odyssey (1968) year = 1968
2001:

In [22]:
joined.groupby('year').agg({'rating': 'mean'}).sort_values('rating', ascending = False)

,rating
year,
1957,4.039535
1954,4.009191
1962,3.969466
1952,3.953125
1964,3.940160
...,...
2000,3.398899
2005,3.359888
1992,3.354943
